In [1]:
# this is hack to import from pre-parent directory
import sys
sys.path.insert(0,'..')
import pandas as pd
import numpy as np
from wing import WingOfEvidence


train_df = pd.read_csv("../datasets/titanic/train.csv",sep=",")
colnames = ["Age"]
X_train = train_df[colnames]
y_train = train_df["Survived"]
wing = WingOfEvidence(n_initial=10,n_target=5,
                        #optimizer="adaptive"
                        optimizer="full-search"
                        )
wing.fit(X_train,y_train)
X_tsrf = wing.transform(X_train)
print(X_tsrf.head())
        #print(X_tsrf["WOE_Age"].value_counts())
        #print(wings.fitted_wing[colnames[0]].get_woe())

D-values in  clear X: 88
Starting WoE calculation per data groups
Current vector type: c
Variable is continiuous
Pre edges dict:
{1: (-inf, 14.0), 2: (14.0, 19.0), 3: (19.0, 22.0), 4: (22.0, 25.0), 5: (25.0, 28.0), 6: (28.0, 31.800000000000068), 7: (31.800000000000068, 36.0), 8: (36.0, 41.0), 9: (41.0, 50.0), 10: (50.0, inf)}
Initial binning:
(-inf, 14.0)    71
(14.0, 19.0)    68
(19.0, 22.0)    65
(22.0, 25.0)    74
(25.0, 28.0)    59
(28.0, 31.8)    91
(31.8, 36.0)    69
(36.0, 41.0)    69
(41.0, 50.0)    74
(50.0, inf)     74
dtype: int64
Searching edges via combinatorical algos
Started search in 381 variants
Search finished, found 57 variants
Got best Gini: 9.391 at variant [-inf  14.  19.  41.  50.  inf]
Current optimal edges dict:
{1: (-inf, 14.0), 2: (14.0, 19.0), 3: (19.0, 41.0), 4: (41.0, 50.0), 5: (50.0, inf)}
WoE groups in fitting after optimal check
3.0    336
1.0     60
4.0     58
5.0     58
2.0     55
Name: woe_group, dtype: int64
     X    y  woe_group       woe
5  NaN  

In [2]:
cdf = pd.DataFrame(np.array([X_train.values.ravel(),y_train.values.ravel()]).T,columns=["X","y"])
cdf = cdf[~pd.isnull(cdf["X"])].copy()

In [13]:
import itertools as it
from pprint import pprint
pre_edges = wing.pre_edges

def generate_layer_variant(df,layer_edges,pre_edges):
    layer_edges_flt = [variant for variant in layer_edges if wing._calc_bins_and_check_mono(df,variant)]
    new_layer = combine_vector(layer_edges_flt,pre_edges)
    return new_layer
def combine_vector(edge_list,base_edges):
    collec_layer = []
    for vect in edge_list:
        high_vect = np.max(vect)
        vect_typ = list(vect)
        idx_v = base_edges>high_vect
        subselected_max = base_edges[idx_v]
        for v in subselected_max:
            if v!= np.inf:
                new_v = vect_typ+[v]
                collec_layer.append(tuple(new_v))
    return collec_layer

f1_layer = [el for el in it.combinations(pre_edges[1:-1], 1)]
f1_layer_flt = [variant for variant in f1_layer if wing._calc_bins_and_check_mono(cdf,variant)]
f2_layer = [el for el in it.combinations(pre_edges[1:-1], 2)]
f2_layer_flt = [variant for variant in f2_layer if wing._calc_bins_and_check_mono(cdf,variant)]
layers_collector = [f1_layer_flt,f2_layer_flt]
iterator_layer = f2_layer_flt
for i in range(wing.n_target):
    lv = generate_layer_variant(cdf,iterator_layer,pre_edges)
    iterator_layer = lv
    print("Current idx: %i"%i)
    pprint(iterator_layer)
    layers_collector.append(lv)
layers_collector = sum(layers_collector,[])
layers_collector

Current idx: 0
[(14.0, 19.0, 22.0),
 (14.0, 19.0, 25.0),
 (14.0, 19.0, 28.0),
 (14.0, 19.0, 31.800000000000068),
 (14.0, 19.0, 36.0),
 (14.0, 19.0, 41.0),
 (14.0, 19.0, 50.0),
 (14.0, 36.0, 41.0),
 (14.0, 36.0, 50.0),
 (14.0, 41.0, 50.0),
 (19.0, 36.0, 41.0),
 (19.0, 36.0, 50.0),
 (19.0, 41.0, 50.0),
 (22.0, 25.0, 28.0),
 (22.0, 25.0, 31.800000000000068),
 (22.0, 25.0, 36.0),
 (22.0, 25.0, 41.0),
 (22.0, 25.0, 50.0),
 (22.0, 28.0, 31.800000000000068),
 (22.0, 28.0, 36.0),
 (22.0, 28.0, 41.0),
 (22.0, 28.0, 50.0),
 (22.0, 36.0, 41.0),
 (22.0, 36.0, 50.0),
 (22.0, 41.0, 50.0),
 (25.0, 36.0, 41.0),
 (25.0, 36.0, 50.0),
 (25.0, 41.0, 50.0),
 (28.0, 36.0, 41.0),
 (28.0, 36.0, 50.0),
 (28.0, 41.0, 50.0),
 (36.0, 41.0, 50.0)]
Current idx: 1
[(14.0, 19.0, 36.0, 41.0),
 (14.0, 19.0, 36.0, 50.0),
 (14.0, 19.0, 41.0, 50.0),
 (22.0, 25.0, 36.0, 41.0),
 (22.0, 25.0, 36.0, 50.0),
 (22.0, 25.0, 41.0, 50.0),
 (22.0, 36.0, 41.0, 50.0),
 (25.0, 36.0, 41.0, 50.0),
 (28.0, 36.0, 41.0, 50.0)]
Current idx: 

[(14.0,),
 (19.0,),
 (22.0,),
 (25.0,),
 (28.0,),
 (31.800000000000068,),
 (36.0,),
 (41.0,),
 (50.0,),
 (14.0, 19.0),
 (14.0, 36.0),
 (14.0, 41.0),
 (14.0, 50.0),
 (19.0, 36.0),
 (19.0, 41.0),
 (19.0, 50.0),
 (22.0, 25.0),
 (22.0, 28.0),
 (22.0, 36.0),
 (22.0, 41.0),
 (22.0, 50.0),
 (25.0, 36.0),
 (25.0, 41.0),
 (25.0, 50.0),
 (28.0, 36.0),
 (28.0, 41.0),
 (28.0, 50.0),
 (36.0, 41.0),
 (36.0, 50.0),
 (41.0, 50.0),
 (14.0, 19.0, 22.0),
 (14.0, 19.0, 25.0),
 (14.0, 19.0, 28.0),
 (14.0, 19.0, 31.800000000000068),
 (14.0, 19.0, 36.0),
 (14.0, 19.0, 41.0),
 (14.0, 19.0, 50.0),
 (14.0, 36.0, 41.0),
 (14.0, 36.0, 50.0),
 (14.0, 41.0, 50.0),
 (19.0, 36.0, 41.0),
 (19.0, 36.0, 50.0),
 (19.0, 41.0, 50.0),
 (22.0, 25.0, 28.0),
 (22.0, 25.0, 31.800000000000068),
 (22.0, 25.0, 36.0),
 (22.0, 25.0, 41.0),
 (22.0, 25.0, 50.0),
 (22.0, 28.0, 31.800000000000068),
 (22.0, 28.0, 36.0),
 (22.0, 28.0, 41.0),
 (22.0, 28.0, 50.0),
 (22.0, 36.0, 41.0),
 (22.0, 36.0, 50.0),
 (22.0, 41.0, 50.0),
 (25.0, 36.0, 

In [36]:
import itertools as it
pre_edges = wing.pre_edges
f1_layer = [el for el in it.combinations(pre_edges[1:-1], 1)]
f1_layer_flt = [variant for variant in f1_layer if wing._calc_bins_and_check_mono(cdf,variant)]
f2_layer = [el for el in it.combinations(pre_edges[1:-1], 2)]
f2_layer_flt = [variant for variant in f2_layer if wing._calc_bins_and_check_mono(cdf,variant)]
f3_layer = combine_vector(f2_layer_flt,pre_edges)
f3_layer_flt = [variant for variant in f3_layer if wing._calc_bins_and_check_mono(cdf,variant)]
f4_layer = combine_vector(f3_layer_flt,pre_edges)
f4_layer_flt = [variant for variant in f4_layer if wing._calc_bins_and_check_mono(cdf,variant)]
f5_layer = combine_vector(f4_layer_flt,pre_edges)
f5_layer_flt = [variant for variant in f5_layer if wing._calc_bins_and_check_mono(cdf,variant)]
f6_layer = combine_vector(f5_layer_flt,pre_edges)
f6_layer_flt = [variant for variant in f6_layer if wing._calc_bins_and_check_mono(cdf,variant)]

In [37]:
all_layers = f2_layer_flt+f3_layer_flt+f4_layer_flt+f5_layer_flt+f6_layer_flt